# 1 Introdução

- Apresentação das técnicas de encoding para variáveis categóricas
- Importância do encoding em modelos de Machine Learning

# 2 Importação das Bibliotecas Necessárias

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from category_encoders import BinaryEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 3 Carregamento e Exploração do Dataset

## 3.1 Descrição do Dataset

Para este exemplo, utilizaremos o famoso dataset "Titanic" disponível no Kaggle. Este dataset contém informações detalhadas sobre os passageiros do RMS Titanic, que afundou em 1912 após colidir com um iceberg. É amplamente utilizado em competições e estudos de machine learning devido à sua riqueza de variáveis e à natureza desafiadora da predição de sobrevivência.

- Principais Características do Dataset:
    - PassengerId: Identificador único para cada passageiro.
    - Survived: Indicador de sobrevivência (0 = Não, 1 = Sim).
    - Pclass: Classe do passageiro (1 = Primeira, 2 = Segunda, 3 = Terceira).
    - Name: Nome do passageiro.
    - Sex: Sexo do passageiro.
    - Age: Idade do passageiro.
    - SibSp: Número de irmãos/cônjuges a bordo.
    - Parch: Número de pais/filhos a bordo.
    - Ticket: Número do bilhete.
    - Fare: Tarifa paga pelo bilhete.
    - Cabin: Número da cabine.
    - Embarked: Porto de embarque (C = Cherbourg, Q = Queenstown, S = Southampton).

- Importância das Variáveis:
    - Classe do Passageiro (Pclass): Indica o nível socioeconômico do passageiro. Passageiros de primeira classe tinham maior probabilidade de sobrevivência.
    - Sexo (Sex): As mulheres tinham uma taxa de sobrevivência significativamente maior do que os homens.
    - Idade (Age): Crianças também tinham uma maior taxa de sobrevivência. A idade ajuda a identificar esses grupos.
    - Número de Irmãos/Cônjuges e Pais/Filhos a Bordo (SibSp e Parch): Essas variáveis ajudam a entender a estrutura familiar a bordo e como ela impactava as chances de sobrevivência.
    - Tarifa (Fare): A tarifa paga pode refletir o status socioeconômico.
    - Porto de Embarque (Embarked): Pode dar insights sobre a origem dos passageiros e possíveis padrões de embarque.

- Fonte do Dataset
    - O dataset está originalmente disponível em: Wine Quality Data Set no UCI Machine Learning Repository
    - Para carregar o dataset em um DataFrame do pandas:

        import pandas as pd

        url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"

        df = pd.read_csv(url, delimiter=';')

## 3.2 Carregamento do Dataset

In [2]:
df = pd.read_csv("datasets/titanic.csv")

# Mostrando as primeiras linhas do dataset
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 3.3 Análise Exploratória

In [3]:
# Visualizando informações gerais sobre o dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
# Visualizando estatísticas descritivas
df.describe(include='all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Dooley, Mr. Patrick",male,NaN,NaN,NaN,347082,NaN,G6,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


# 4 Lidando com dados faltantes

In [5]:
# Removendo colunas desnecessárias e linhas com valores nulos
df_clean = df.drop(columns=['Name', 'Ticket', 'Cabin', 'PassengerId']).dropna()

In [6]:
df_clean.describe(include='all')

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,712.000000,712.000000,712,712.000000,712.000000,712.000000,712.000000,712
unique,NaN,NaN,2,NaN,NaN,NaN,NaN,3
top,NaN,NaN,male,NaN,NaN,NaN,NaN,S
freq,NaN,NaN,453,NaN,NaN,NaN,NaN,554
mean,0.404494,2.240169,NaN,29.642093,0.514045,0.432584,34.567251,NaN
std,0.491139,0.836854,NaN,14.492933,0.930692,0.854181,52.938648,NaN
min,0.000000,1.000000,NaN,0.420000,0.000000,0.000000,0.000000,NaN
25%,0.000000,1.000000,NaN,20.000000,0.000000,0.000000,8.050000,NaN
50%,0.000000,2.000000,NaN,28.000000,0.000000,0.000000,15.645850,NaN
75%,1.000000,3.000000,NaN,38.000000,1.000000,1.000000,33.000000,NaN


# 5 Técnicas de Encoding para Variáveis Categóricas

## 5.1 One-Hot Encoding

In [7]:
df_clean.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [8]:
one_hot_encoder = OneHotEncoder(sparse_output=False, drop='first')
one_hot_encoded = one_hot_encoder.fit_transform(df_clean[['Sex', 'Embarked']])

In [9]:
# Convertendo o resultado para um DataFrame
one_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=one_hot_encoder.get_feature_names_out(['Sex', 'Embarked']))
one_hot_encoded_df.head()

,Sex_male,Embarked_Q,Embarked_S
0,1.0,0.0,1.0
1,0.0,0.0,0.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,1.0,0.0,1.0


# 5.2 Ordinal Encoding

In [10]:
ordinal_encoder = OrdinalEncoder()
ordinal_encoded = ordinal_encoder.fit_transform(df_clean[['Pclass', 'Embarked']])

In [11]:
# Convertendo o resultado para um DataFrame
ordinal_encoded_df = pd.DataFrame(ordinal_encoded, columns=['Pclass_ordinal', 'Embarked_ordinal'])
ordinal_encoded_df.head()

,Pclass_ordinal,Embarked_ordinal
0,2.0,2.0
1,0.0,0.0
2,2.0,2.0
3,0.0,2.0
4,2.0,2.0


# 5.3 Binary Encoding

In [12]:
binary_encoder = BinaryEncoder()
binary_encoded = binary_encoder.fit_transform(df_clean[['Sex', 'Embarked']])

In [13]:
# Mostrando as primeiras linhas do dataset codificado
binary_encoded.head()

,Sex_0,Sex_1,Embarked_0,Embarked_1
0,0,1,0,1
1,1,0,1,0
2,1,0,0,1
3,1,0,0,1
4,0,1,0,1


# 6 Comparação de Desempenho em Modelos de Machine Learning

## 6.1 Preparação dos Dados

In [14]:
# Definindo variáveis dependentes e independentes
X_raw = df_clean.drop(columns='Survived')
y = df_clean['Survived']

# Aplicando One-Hot Encoding
X_one_hot = pd.concat([df_clean.drop(columns=['Sex', 'Embarked']).reset_index(drop=True), one_hot_encoded_df.reset_index(drop=True)], axis=1)

# Aplicando Ordinal Encoding
X_ordinal = df_clean.drop(columns=['Sex', 'Embarked'])
X_ordinal[['Pclass', 'Embarked']] = ordinal_encoded_df

# Aplicando Binary Encoding
X_binary = pd.concat([df_clean.drop(columns=['Sex', 'Embarked']), binary_encoded], axis=1)

## 6.2 Treinamento e Avaliação dos Modelos

In [15]:
def train_and_evaluate(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print(f"Acurácia: {accuracy_score(y_test, y_pred)}")
    print("Matriz de Confusão:")
    print(confusion_matrix(y_test, y_pred))
    print("Relatório de Classificação:")
    print(classification_report(y_test, y_pred))

In [16]:
# Treinando e avaliando modelos com diferentes técnicas de encoding
print("Desempenho com Dados Brutos:")
train_and_evaluate(X_raw.select_dtypes(include=['number']), y)

Desempenho com Dados Brutos:
Acurácia: 0.6542056074766355
Matriz de Confusão:
[[87 35]
 [39 53]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.69      0.71      0.70       122
           1       0.60      0.58      0.59        92

    accuracy                           0.65       214
   macro avg       0.65      0.64      0.65       214
weighted avg       0.65      0.65      0.65       214



In [17]:
print("\nDesempenho com One-Hot Encoding:")
train_and_evaluate(X_one_hot, y)


Desempenho com One-Hot Encoding:
Acurácia: 1.0
Matriz de Confusão:
[[122   0]
 [  0  92]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       122
           1       1.00      1.00      1.00        92

    accuracy                           1.00       214
   macro avg       1.00      1.00      1.00       214
weighted avg       1.00      1.00      1.00       214



In [18]:
print("\nDesempenho com Ordinal Encoding:")
train_and_evaluate(X_ordinal, y)


Desempenho com Ordinal Encoding:
Acurácia: 1.0
Matriz de Confusão:
[[122   0]
 [  0  92]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       122
           1       1.00      1.00      1.00        92

    accuracy                           1.00       214
   macro avg       1.00      1.00      1.00       214
weighted avg       1.00      1.00      1.00       214



In [19]:
print("\nDesempenho com Binary Encoding:")
train_and_evaluate(X_binary, y)


Desempenho com Binary Encoding:
Acurácia: 1.0
Matriz de Confusão:
[[122   0]
 [  0  92]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       122
           1       1.00      1.00      1.00        92

    accuracy                           1.00       214
   macro avg       1.00      1.00      1.00       214
weighted avg       1.00      1.00      1.00       214

